In [4]:
import pandas as pd
import numpy as np
import math


In [5]:
df = pd.read_csv('google-cluster-data-1.csv',sep=' ')

In [6]:
df

,Time,ParentID,TaskID,JobType,NrmlTaskCores,NrmlTaskMem,Unnamed: 6
0,90000,757745334,1488529826,0,0.000000,0.031130,NaN
1,90000,975992247,1488529821,0,0.000000,0.000000,NaN
2,90000,1468458091,1488529832,1,0.021875,0.002353,NaN
3,90000,1460281235,1488529840,0,0.000000,0.000000,NaN
4,90000,1164728954,1488529835,0,0.003125,0.001638,NaN
...,...,...,...,...,...,...,...
3535024,112500,1487094655,1487103476,0,0.000000,0.000879,NaN
3535025,112500,1461321601,1465612301,0,0.000000,0.000879,NaN
3535026,112500,1487094655,1487097223,0,0.000000,0.000879,NaN
3535027,112500,618817162,1485932004,1,0.000000,0.000879,NaN


## Append ExecutionTime column to dataframe

In [7]:
exec_time_arr = []
subtasks = 0
quantum = 300
for task in df.itertuples():
    taskID = task.TaskID
    burstTime = ((taskID%2)+1)*quantum      # only two options for burst time
    exec_time_arr.append(burstTime)
    subtasks += burstTime / quantum

exec_time_frame = pd.DataFrame({'ExecutionTime': exec_time_arr})
df = df.join(exec_time_frame)
print("Subtasks:", subtasks)
df

,Time,ParentID,TaskID,JobType,NrmlTaskCores,NrmlTaskMem,Unnamed: 6,ExecutionTime
0,90000,757745334,1488529826,0,0.000000,0.031130,NaN,300
1,90000,975992247,1488529821,0,0.000000,0.000000,NaN,600
2,90000,1468458091,1488529832,1,0.021875,0.002353,NaN,300
3,90000,1460281235,1488529840,0,0.000000,0.000000,NaN,300
4,90000,1164728954,1488529835,0,0.003125,0.001638,NaN,600
...,...,...,...,...,...,...,...,...
3535024,112500,1487094655,1487103476,0,0.000000,0.000879,NaN,300
3535025,112500,1461321601,1465612301,0,0.000000,0.000879,NaN,600
3535026,112500,1487094655,1487097223,0,0.000000,0.000879,NaN,600
3535027,112500,618817162,1485932004,1,0.000000,0.000879,NaN,300


## Create 100 VMs
Each of them has 7 CPU units and 11 MEM units. You could choose to use a 100*2 matrix to represent this data structure. First column is CPU and second column is MEM.

# Greedy Algorithm 3 ways
* Optimize energy consumption
* Optimize cost
* Optimize turn around time


In [9]:
# Create power and cost dictionaries
Pwr_dy_t = {}
Pwr_st_t = {}
Pwr_t = {}
Cost_t = {}

Price_t = [0.5, 0.5, 0.6, 0.6, 0.6, 0.7, 0.7, 0.6, 0.6, 0.8, 0.8, 0.8, 0.8]
totalCost = 0
totalPwr = 0

for time in range(90000,113100,300):
    Pwr_dy_t[time] = []
    Pwr_st_t[time] = []
    Pwr_t[time] = []
    Cost_t[time] = []

In [ ]:
def powerCalculator(vms, task_idx, vm_idx):
  c = 5
  a = 100
  b = 200
  dy_pwr_threshold = 0.5
  
  pwr_dy = 0
  pwr_st = 0

  pwr_curr = 0
  pwr_next = 0
  
  cpu = df['NrmlTaskCores'][task_idx]

  # current vm calculation

  ccr = vms[0][vm_idx]  # current core remaining 
  usage_rate_curr = 1 - (ccr/25)    # instead of 25, maybe put cpu variable? unsure

  if(usage_rate_curr > 0):
    pwr_st = c
  if(usage_rate_curr < dy_pwr_threshold):
    pwr_dy = a*threshold
  else:
    pwr_dy = (a * threshold) + b*(usage_rate_curr - dy_pwr_threshold)*(usage_rate_curr - dy_pwr_threshold)
  pwr_curr = pwr_st + pwr_dy

  # next vm calculation

  ncr = vms[0][vm_idx] - cpu    # next core remaining
  usage_rate_next = 1 - (ncr/25)
  pwr_st = 0    # reset pwr_st because will reuse this var
  if(usage_rate_next > 0):
    pwr_st = c
  if(usage_rate_next < dy_pwr_threshold):
    pwr_dy = a*threshold
  else:
    pwr_dy = (a * threshold) + b*(usage_rate_next - dy_pwr_threshold)*(usage_rate_next - dy_pwr_threshold)
  pwr_next = pwr_st + pwr_dy


  return pwr_next - pwr_curr

In [ ]:
def greedy(task_queue, vms):
    reject_queue = []
    available_vms = []
    for i in task_queue:
        vm_id = i % 100
        # cpu, mem = df['NrmlTaskCores'][i], df['NrmlTaskMem'][i]

        while (df['NrmlTaskCores'][i] <= vms[0][vm_id]) and (df['NrmlTaskMem'][i] <= vms[1][vm_id]):
            # add this vm to a list of vms that work for this task
            # find all that work, and calculate power consumption, cost, turnaround time and optimize
            # take the power consumption at current time t, save previous power and compare the increment
            # optimize the change in each of the 3 vars
            available_vms.append(vm_id)
            # vms[vm_id][0] = vms[vm_id][0] - df['NrmlTaskCores'][i]   # subtract out used cpus for this task 
            # vms[vm_id][1] = vms[vm_id][1] - df['NrmlTaskMem'][i]   # subtract out used mem units for this task
        # else search for next one
        # different greeedy algs



In [ ]:
def createTaskQueues(taskInfo):
    
    taskQueues = {}
    for time in range(90000,115500,300):
        taskQueues[time] = []
    time = 90000
    for taskIdx in range(len(taskInfo['Time'])):
        if(taskInfo['Time'][taskIdx] == time):
            exTime = taskInfo['ExecutionTime'][taskIdx]
            timeCnt = 0
            while(exTime > 0):
                taskQueues[time+timeCnt].append(taskIdx)
                timeCnt = timeCnt + 300
                exTime = exTime - 300
        else:
            time = time + 300
    return taskQueues